<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/ADAPMIT/CPU_ADAPMIT_ClimateBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 13:29:13] [setup] RAM Tracking...
[codecarbon INFO @ 13:29:13] [setup] GPU Tracking...
[codecarbon INFO @ 13:29:13] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:29:13] [setup] CPU Tracking...
[codecarbon WARNING @ 13:29:13] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:29:15] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 13:29:15] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:29:15] >>> Tracker's metadata:
[codecarbon INFO @ 13:29:15]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 13:29:15]   Python version: 3.10.12
[codecarbon INFO @ 13:29:15]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 13:29:15]   Available RAM : 12.675 GB
[codecarbon INFO @ 13:29:15]   CPU count: 2
[codecarbon INFO @ 13:29:15]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:29:15]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

[codecarbon INFO @ 13:29:45] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 13:29:45] Energy consumed for all GPUs : 0.000091 kWh. Total GPU Power : 10.836049340327072 W
[codecarbon INFO @ 13:29:45] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:29:45] 0.000485 kWh of electricity used since the beginning.


# Manual Inputs


In [5]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"


### Define labels ##################
label_names= ['AdaptationLabel','MitigationLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "climatebert/distilroberta-base-climate-f"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "ADAPMIT-multilabel-climatebert"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=6.03e-5
batch_size = 16
num_train_epochs= 5
weight_decay=0.05
warmup_steps = 300
gradient_accumulation_steps = 1

# Automated

## Dataset

In [6]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

[codecarbon INFO @ 13:30:00] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 13:30:00] Energy consumed for all GPUs : 0.000135 kWh. Total GPU Power : 10.774864436275571 W
[codecarbon INFO @ 13:30:00] Energy consumed for all CPUs : 0.000532 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:30:00] 0.000727 kWh of electricity used since the beginning.


{0: 'AdaptationLabel', 1: 'MitigationLabel'}


In [7]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 12538
0 . AdaptationLabel : 5439
1 . MitigationLabel : 6659

 Classes Representation in Test Dataset: 1190
0 . AdaptationLabel : 533
1 . MitigationLabel : 604


## Model

In [8]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 12538
Validation data: 1190


Map:   0%|          | 0/12538 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

Map:   0%|          | 0/12538 [00:00<?, ? examples/s]

[codecarbon INFO @ 13:30:15] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 13:30:15] Energy consumed for all GPUs : 0.000221 kWh. Total GPU Power : 20.365413218127102 W
[codecarbon INFO @ 13:30:15] Energy consumed for all CPUs : 0.000710 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:30:15] 0.001010 kWh of electricity used since the beginning.


Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

## Metrics

In [10]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [11]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

# Training

In [12]:
multi_trainer =  Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 13:30:25] [setup] RAM Tracking...
[codecarbon INFO @ 13:30:25] [setup] GPU Tracking...
[codecarbon INFO @ 13:30:25] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:30:25] [setup] CPU Tracking...
[codecarbon WARNING @ 13:30:25] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:30:26] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 13:30:26] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:30:26] >>> Tracker's metadata:
[codecarbon INFO @ 13:30:26]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 13:30:26]   Python version: 3.10.12
[codecarbon INFO @ 13:30:26]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 13:30:26]   Available RAM : 12.675 GB
[codecarbon INFO @ 13:30:26]   CPU count: 2
[codecarbon INFO @ 13:30:26]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 13:30:26]   GPU c

In [ ]:
multi_trainer.train()

In [14]:
tracker.stop()

[codecarbon INFO @ 14:10:11] Energy consumed for RAM : 0.003237 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 14:10:11] Energy consumed for all GPUs : 0.046212 kWh. Total GPU Power : 33.09819801086527 W
[codecarbon INFO @ 14:10:11] Energy consumed for all CPUs : 0.028977 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:10:11] 0.078426 kWh of electricity used since the beginning.


0.03826714052400342

In [15]:
multi_trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

events.out.tfevents.1709645429.0c46e3d01814.525.0:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/ADAPMIT-multilabel-climatebert/commit/39b22d82ec565dcd209ee6254fbbc75ee367352d', commit_message='End of training', commit_description='', oid='39b22d82ec565dcd209ee6254fbbc75ee367352d', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.3512,1.849697,0.000058,1.0,784,0.325310,0.853041,0.827311,0.857213,0.888303,0.831092,0.888303,0.870315,0.823810,0.870326,15.1964,78.308,4.935
1,0.2152,3.321303,0.000044,2.0,1568,0.260418,0.899913,0.857983,0.900201,0.909411,0.852101,0.909411,0.904637,0.850980,0.904635,15.1220,78.694,4.960
2,0.1348,6.147693,0.000024,3.0,2352,0.290795,0.903813,0.862605,0.905911,0.917326,0.858824,0.917326,0.910519,0.856583,0.910734,14.9575,79.559,5.014
3,0.0767,3.565788,0.000007,4.0,3136,0.336715,0.899914,0.856303,0.900038,0.917326,0.858824,0.917326,0.908537,0.852381,0.908541,14.9813,79.433,5.006
4,0.0475,3.439491,0.000000,5.0,3920,0.353537,0.899914,0.855882,0.900092,0.917326,0.859244,0.917326,0.908537,0.852101,0.908540,15.0277,79.187,4.991


In [ ]:
# import json
# with open("ADAPMIT_climatebert_logs.json", "w") as fp:
#     json.dump(log_results , fp)

# Predict and Evaluate

## Trainer

In [17]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [18]:
y_true = y_true.bool()
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

AdaptationLabel
[[609  48]
 [ 49 484]] 

MitigationLabel
[[518  68]
 [ 45 559]] 



In [19]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,AdaptationLabel,0.909774,0.908068,0.908920,533.0
1,MitigationLabel,0.891547,0.925497,0.908205,604.0
2,micro avg,0.899914,0.917326,0.908537,1137.0
3,macro avg,0.900661,0.916782,0.908562,1137.0
4,weighted avg,0.900092,0.917326,0.908540,1137.0
5,samples avg,0.855882,0.859244,0.852101,1137.0
